**HW5: Image Classification**

**GOAL:** The goal in this assignment is similar to the goal of the previous assignment (Homework 4) in which we
trained FCNs to classify images in the FashionMNIST data set. In this assignment, however, we have a
budget of weights that you can incorporate into your neural network model and we will compare FCNs vs.
Convolutional Neural Networks (CNNs).

**Task 1:** *With the constraint of incorporating up to 100K weights in the design of your FCN model, perform
hyperparameter tuning to achieve FCN model whose testing classification accuracy is above 88% on
the testing set*


In [18]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import FashionMNIST
from sklearn.model_selection import train_test_split

In [19]:
# Use the following code to load and normalize the dataset for training and testing
# It will downlad the dataset into data subfolder (change to your data folder name)
train_dataset = torchvision.datasets.FashionMNIST('C:\\Users\\Sarayu G\\582\\', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))

test_dataset = torchvision.datasets.FashionMNIST('C:\\Users\\Sarayu G\\582\\', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))


# Use the following code to create a validation set of 10%
train_indices, val_indices, _, _ = train_test_split(
    range(len(train_dataset)),
    train_dataset.targets,
    stratify=train_dataset.targets,
    test_size=0.1,
)

# Generate training and validation subsets based on indices
train_split = Subset(train_dataset, train_indices)
val_split = Subset(train_dataset, val_indices)


# set batches sizes
train_batch_size = 900 #Define train batch size
test_batch_size  = 1000 #Define test batch size (can be larger than train batch size)


# Define dataloader objects that help to iterate over batches and samples for
# training, validation and testing
train_batches = DataLoader(train_split, batch_size=train_batch_size, shuffle=True)
val_batches = DataLoader(val_split, batch_size=train_batch_size, shuffle=True)
test_batches = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=True)
                                           
num_train_batches=len(train_batches)
num_val_batches=len(val_batches)
num_test_batches=len(test_batches)


#print(num_train_batches)
#print(num_val_batches)
#print(num_test_batches)

In [20]:
# Define the FCN model class
class FCN(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dims):
        super(FCN, self).__init__()
        self.hidden_layers = nn.ModuleList([nn.Linear(input_dim, hidden_dims[0])])
        for i in range(len(hidden_dims) - 1):
            self.hidden_layers.append(nn.Linear(hidden_dims[i], hidden_dims[i + 1]))
        self.output_layer = nn.Linear(hidden_dims[-1], output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        for layer in self.hidden_layers:
            x = self.relu(layer(x))
        x = self.output_layer(x)
        return x

In [21]:
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
import random
import time

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Initialize data dimensions and hyperparameters
input_dim = 784
output_dim = 10
hidden_dims = [100, 64]  # hidden layer configuration
learning_rate = 0.003
epochs = 15
max_weights = 100000

# Load FashionMNIST data and define train_batches, val_batches

# Define the FCN model
model = FCN(input_dim=input_dim, output_dim=output_dim, hidden_dims=hidden_dims)

# Count total parameters in the model
total_params = sum(p.numel() for p in model.parameters())
print("Total parameters in the model:", total_params)

# Define loss function and optimizer
loss_func = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_loss_list = np.zeros((epochs,))
validation_loss_list = np.zeros((epochs,))
validation_accuracy_list = np.zeros((epochs,))
test_accuracy_list = []

start_time = time.time()

# Iterate over epochs and train the FCN model
for epoch in tqdm(range(epochs)):
    model.train()
    epoch_loss = 0.0
    num_batches = 0
    for train_features, train_labels in train_batches:
        optimizer.zero_grad()
        train_features = train_features.reshape(-1, input_dim)
        outputs = model(train_features)
        loss = loss_func(outputs, train_labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        num_batches += 1
        
    average_epoch_loss = epoch_loss / num_batches
    train_loss_list[epoch] = average_epoch_loss

    # Evaluate validation accuracy
    val_loss = 0
    val_acc = 0
    total_samples = 0
    for val_features, val_labels in val_batches:
        with torch.no_grad():
            model.eval()
            val_features = val_features.reshape(-1, 28*28)
            val_outputs = model(val_features)
            loss = loss_func(val_outputs, val_labels)
            val_loss += loss.item()
            _, predicted = torch.max(val_outputs.data, 1)
            val_acc += (predicted == val_labels).sum().item()
            total_samples += val_labels.size(0)
    average_val_loss = val_loss / len(val_batches)
    average_val_acc = val_acc / total_samples * 100
    # Record average validation loss and accuracy for the epoch
    validation_loss_list[epoch] = average_val_loss
    validation_accuracy_list[epoch] = average_val_acc

    #print(f"Epoch {epoch + 1}/{epochs}, Validation Accuracy: {average_val_acc}%")

    # Check if the total parameters are within the budget
    if total_params > max_weights:
        break

end_time = time.time()
training_time = end_time - start_time

# Print total training time
#print("Total training time:", training_time, "seconds")

Total parameters in the model: 85614


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [03:50<00:00, 15.38s/it]


In [22]:
# Initialize variables for computing accuracy
total_correct = 0
total_samples = 0

# Telling PyTorch we aren't passing inputs to network for training purpose
with torch.no_grad():
    for test_features, test_labels in test_batches:
        model.eval()
        
        # Reshape test images into a vector
        test_features = test_features.reshape(-1, 28*28)
        
        # Compute test outputs (targets)
        test_outputs = model(test_features)
        
        # Compute predicted labels
        _, predicted = torch.max(test_outputs, 1)
        
        # Compute number of correct predictions in the batch
        total_correct += (predicted == test_labels).sum().item()
        
        # Count total number of samples in the batch
        total_samples += test_labels.size(0)
        # Compute total accuracy
        test_accuracy = total_correct / total_samples * 100
        #print("Test Accuracy:", test_accuracy, "%")
        test_accuracy_list.append(test_accuracy)

test_accuracy_array = np.array(test_accuracy_list)
test_accuracy_std = np.std(test_accuracy_array)

# Calculate upper and lower bounds of testing accuracy
test_accuracy_upper_bound = np.max(test_accuracy_array)
test_accuracy_lower_bound = np.min(test_accuracy_array)

#print("Standard Deviation of Testing Accuracy:", test_accuracy_std)
#print("Upper Bound of Testing Accuracy:", test_accuracy_upper_bound)
#print("Lower Bound of Testing Accuracy:", test_accuracy_lower_bound)

In [23]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set Seaborn style and font scale
sns.set_theme(style='whitegrid', font_scale=1.5)

#fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))

# Plot training loss
#axes[0].plot(train_loss_list, linewidth=3)
#axes[0].set_ylabel("Training Loss")
#axes[0].set_xlabel("Epochs")

# Plot validation accuracy
#axes[1].plot(validation_accuracy_list, linewidth=3, color='gold')
#axes[1].set_ylabel("Validation Accuracy")
#axes[1].set_xlabel("Epochs")

#sns.despine()

# Display the plots
#plt.show()

In [24]:
#fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))

#axes[0].plot(train_loss_list, label='Training loss', linewidth=3)
#axes[0].plot(validation_loss_list, label='Validation loss', linewidth=3)
#axes[0].set_title('Training Loss vs Validation loss')
#axes[0].set_xlabel('Epochs')
#axes[0].set_ylabel('loss')
#axes[0].legend()

**Task 2:** *Reduce by half the number of weights in the FCN 100K model to create FCN 50K, and also double
the number of weights to create FCN 200K. Train these models similarly to FCN 100K and study the
accuracy of these models by comparing the different FCN variants.*

For 50K weights:

In [25]:
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Initialize data dimensions and hyperparameters
input_dim = 784
output_dim = 10
hidden_dims = [54, 54]  # hidden layer configuration
learning_rate = 0.003
epochs = 15
max_weights = 50000

# Load FashionMNIST data and define train_batches, val_batches

# Define the FCN model
model = FCN(input_dim=input_dim, output_dim=output_dim, hidden_dims=hidden_dims)

# Count total parameters in the model
total_params = sum(p.numel() for p in model.parameters())
print("Total parameters in the model:", total_params)

# Define loss function and optimizer
loss_func = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_loss_list = np.zeros((epochs,))
validation_loss_list = np.zeros((epochs,))
validation_accuracy_list = np.zeros((epochs,))
test_accuracy = 0

start_time = time.time()

# Iterate over epochs and train the FCN model
for epoch in tqdm(range(epochs)):
    model.train()
    epoch_loss = 0.0
    num_batches = 0
    for train_features, train_labels in train_batches:
        optimizer.zero_grad()
        train_features = train_features.reshape(-1, input_dim)
        outputs = model(train_features)
        loss = loss_func(outputs, train_labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        num_batches += 1
        
    average_epoch_loss = epoch_loss / num_batches
    train_loss_list[epoch] = average_epoch_loss

    # Evaluate validation accuracy
    val_loss = 0
    val_acc = 0
    total_samples = 0
    for val_features, val_labels in val_batches:
        with torch.no_grad():
            model.eval()
            val_features = val_features.reshape(-1, 28*28)
            val_outputs = model(val_features)
            loss = loss_func(val_outputs, val_labels)
            val_loss += loss.item()
            _, predicted = torch.max(val_outputs.data, 1)
            val_acc += (predicted == val_labels).sum().item()
            total_samples += val_labels.size(0)
    average_val_loss = val_loss / len(val_batches)
    average_val_acc = val_acc / total_samples * 100
    # Record average validation loss and accuracy for the epoch
    validation_loss_list[epoch] = average_val_loss
    validation_accuracy_list[epoch] = average_val_acc

    #print(f"Epoch {epoch + 1}/{epochs}, Validation Accuracy: {average_val_acc}%")

    # Check if the total parameters are within the budget
    if total_params > max_weights:
        break

end_time = time.time()
training_time = end_time - start_time

# Print total training time
#print("Total training time:", training_time, "seconds")

Total parameters in the model: 45910


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [03:30<00:00, 14.02s/it]


In [26]:
# Initialize variables for computing accuracy
total_correct = 0
total_samples = 0

# Telling PyTorch we aren't passing inputs to network for training purpose
with torch.no_grad():
    for test_features, test_labels in test_batches:
        model.eval()
        
        # Reshape test images into a vector
        test_features = test_features.reshape(-1, 28*28)
        
        # Compute test outputs (targets)
        test_outputs = model(test_features)
        
        # Compute predicted labels
        _, predicted = torch.max(test_outputs, 1)
        
        # Compute number of correct predictions in the batch
        total_correct += (predicted == test_labels).sum().item()
        
        # Count total number of samples in the batch
        total_samples += test_labels.size(0)

# Compute total accuracy
test_accuracy = total_correct / total_samples * 100

# Report total accuracy
#print("Test Accuracy:", test_accuracy, "%")

In [27]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set Seaborn style and font scale
sns.set_theme(style='whitegrid', font_scale=1.5)

# Create subplots
#fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))

# Plot training loss
#axes[0].plot(train_loss_list, linewidth=3)
#axes[0].set_ylabel("Training Loss")
#axes[0].set_xlabel("Epochs")

# Plot validation accuracy
#axes[1].plot(validation_accuracy_list, linewidth=3, color='gold')
#axes[1].set_ylabel("Validation Accuracy")
#axes[1].set_xlabel("Epochs")

# Remove the top and right spines from the plots
#sns.despine()

# Display the plots
#plt.show()

In [28]:
#fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))

#axes[0].plot(train_loss_list, label='Training loss', linewidth=3)
#axes[0].plot(validation_loss_list, label='Validation loss', linewidth=3)
#axes[0].set_title('Training Loss vs Validation loss')
#axes[0].set_xlabel('Epochs')
#axes[0].set_ylabel('loss')
#axes[0].legend()

**Task 2:**

For 200K weights:

In [29]:
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Initialize data dimensions and hyperparameters
input_dim = 784
output_dim = 10
hidden_dims = [200, 184]  # hidden layer configuration
learning_rate = 0.003
epochs = 15
max_weights = 200000

# Load FashionMNIST data and define train_batches, val_batches

# Define the FCN model
model = FCN(input_dim=input_dim, output_dim=output_dim, hidden_dims=hidden_dims)

# Count total parameters in the model
total_params = sum(p.numel() for p in model.parameters())
print("Total parameters in the model:", total_params)

# Define loss function and optimizer
loss_func = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_loss_list = np.zeros((epochs,))
validation_loss_list = np.zeros((epochs,))
validation_accuracy_list = np.zeros((epochs,))
test_accuracy = 0

start_time = time.time()

# Iterate over epochs and train the FCN model
for epoch in tqdm(range(epochs)):
    model.train()
    epoch_loss = 0.0
    num_batches = 0
    for train_features, train_labels in train_batches:
        optimizer.zero_grad()
        train_features = train_features.reshape(-1, input_dim)
        outputs = model(train_features)
        loss = loss_func(outputs, train_labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        num_batches += 1
        
    average_epoch_loss = epoch_loss / num_batches
    train_loss_list[epoch] = average_epoch_loss

    # Evaluate validation accuracy
    val_loss = 0
    val_acc = 0
    total_samples = 0
    for val_features, val_labels in val_batches:
        with torch.no_grad():
            model.eval()
            val_features = val_features.reshape(-1, 28*28)
            val_outputs = model(val_features)
            loss = loss_func(val_outputs, val_labels)
            val_loss += loss.item()
            _, predicted = torch.max(val_outputs.data, 1)
            val_acc += (predicted == val_labels).sum().item()
            total_samples += val_labels.size(0)
    average_val_loss = val_loss / len(val_batches)
    average_val_acc = val_acc / total_samples * 100
    # Record average validation loss and accuracy for the epoch
    validation_loss_list[epoch] = average_val_loss
    validation_accuracy_list[epoch] = average_val_acc

    #print(f"Epoch {epoch + 1}/{epochs}, Validation Accuracy: {average_val_acc}%")

    # Check if the total parameters are within the budget
    if total_params > max_weights:
        break

end_time = time.time()
training_time = end_time - start_time

# Print total training time
#print("Total training time:", training_time, "seconds")

Total parameters in the model: 195834


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [03:39<00:00, 14.64s/it]


In [30]:
# Initialize variables for computing accuracy
total_correct = 0
total_samples = 0

# Telling PyTorch we aren't passing inputs to network for training purpose
with torch.no_grad():
    for test_features, test_labels in test_batches:
        model.eval()
        
        # Reshape test images into a vector
        test_features = test_features.reshape(-1, 28*28)
        
        # Compute test outputs (targets)
        test_outputs = model(test_features)
        
        # Compute predicted labels
        _, predicted = torch.max(test_outputs, 1)
        
        # Compute number of correct predictions in the batch
        total_correct += (predicted == test_labels).sum().item()
        
        # Count total number of samples in the batch
        total_samples += test_labels.size(0)

# Compute total accuracy
test_accuracy = total_correct / total_samples * 100

# Report total accuracy
#print("Test Accuracy:", test_accuracy, "%")

In [31]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set Seaborn style and font scale
sns.set_theme(style='whitegrid', font_scale=1.5)

# Create subplots
#fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))

# Plot training loss
#axes[0].plot(train_loss_list, linewidth=3)
#axes[0].set_ylabel("Training Loss")
#axes[0].set_xlabel("Epochs")

# Plot validation accuracy
#axes[1].plot(validation_accuracy_list, linewidth=3, color='gold')
#axes[1].set_ylabel("Validation Accuracy")
#axes[1].set_xlabel("Epochs")

# Remove the top and right spines from the plots
#sns.despine()

# Display the plots
#plt.show()

In [32]:
#fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))

#axes[0].plot(train_loss_list, label='Training loss', linewidth=3)
#axes[0].plot(validation_loss_list, label='Validation loss', linewidth=3)
#axes[0].set_title('Training Loss vs Validation loss')
#axes[0].set_xlabel('Epochs')
#axes[0].set_ylabel('loss')
#axes[0].legend()